In [ ]:
from ingest.linkedin import linkedin_client

linkedin_client


## Linked-in Voyager dev

In [ ]:
# manually retrieve html? - no new data, same voyager endpoints
import requests
from bs4 import BeautifulSoup

url = "https://www.linkedin.com/in/samhardyhey/recent-activity/reactions/"
response = linkedin_client.client.session.get(url)

if response.status_code == 200:
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
else:
    print(f"Error {response.status_code}: {response.reason}")


In [ ]:
# parse voyager endpoints
from bs4 import BeautifulSoup

voyager_endpoints = []
with open("./data/selenium_scrape.html", "r") as f:
    soup = BeautifulSoup(f, "html.parser")
    for code in soup.find_all("code", {"id": lambda x: x and x.startswith("datalet-")}):
        voyager_endpoints.append(eval(code.text.strip()))


In [ ]:
# inspect each endpoint's response
res_all = []
for endpoint in voyager_endpoints:
    print(endpoint["request"])
    replace = "/voyager/api"
    res = linkedin_client._fetch(uri=endpoint["request"].replace(replace, ""))
    res = res.json()
    res["endpoint"] = endpoint["request"]
    res_all.append(res)


In [ ]:
import srsly

srsly.write_jsonl("./data/voyager_responses.json", res_all)


In [ ]:
from ingest.linkedin import Linkedin

# TODO: find liked post resource? > inspect page element
# https://www.linkedin.com/in/samhardyhey/recent-activity/reactions/
x = {
    "request": "/voyager/api/me",
    "status": 200,
    "body": "bpr-guid-696917",
    "method": "GET",
    "headers": {"x-li-uuid": "AAX36V11Xzkme9nvkQAg7g\u003D\u003D"},
}
x = {
    "request": "/voyager/api/growth/pageContent/flagship_desktop_surveys?slotId\u003Dsurveys",
    "status": 200,
    "body": "bpr-guid-106668",
    "method": "GET",
    "headers": {"x-li-uuid": "AAX4ENQwMUo+0Bta3nH4dg\u003D\u003D"},
}


## evaluate via Argilla

In [70]:
from argilla.client.api import load

# after some annotations, load in dataset
labelled = (
    load("tech_classification_validation").to_pandas().pipe(lambda x: x[~x.annotation.isna()])
    .assign(prediction=lambda x: x.prediction.apply(lambda y: y[0][0]))
)

from sklearn.metrics import classification_report

pd.DataFrame(classification_report(labelled.annotation, labelled.prediction, output_dict=True))

<Future at 0x13f988a00 state=pending>

Output()